In [80]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [81]:
#Import MNIST Data
from tensorflow.examples.tutorials.mnist import input_data

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
full_data_x = mnist.train.images

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [82]:
#Parameters 
num_steps = 50 #Total steps to train
batch_size = 1024 #Nums of samples per batch
k = 25 # Clusters num
num_classes = 10 # 10 digits in MNIST
num_features = 784 # Each Image : 28 * 28 pixels

# Input Images
X = tf.placeholder(tf.float32, shape = [None, num_features])

# Labels (for assigning a label to  a centriod and testing)
Y = tf.placeholder(tf.float32, shape = [None, num_classes])

# K-Means Parameters
kmeans = KMeans(inputs = X, num_clusters = k, distance_metric = 'cosine', use_mini_batch = True)

In [83]:
# Build KMeans graph
(all_scores, cluster_idx, scores, cluster_centers_initialized,
 init_op, train_op) = kmeans.training_graph()

In [84]:
cluster_idx = cluster_idx[0] # fix for cluster_idx being a tuple
avg_distance = tf.reduce_mean(scores)

In [85]:
# Initialization of the variables (Assign default value)
init_vars = tf.global_variables_initializer()

In [86]:
# Start TF Session
sess = tf.Session()

# Run Initializer
sess.run(init_vars, feed_dict = {X : full_data_x})
sess.run(init_op, feed_dict = {X : full_data_x})

True

In [87]:
# Training 
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                        feed_dict = {X : full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Average Distance: %f" %(i, d))

Step 1, Average Distance: 0.341471
Step 10, Average Distance: 0.221609
Step 20, Average Distance: 0.220328
Step 30, Average Distance: 0.219776
Step 40, Average Distance: 0.219419
Step 50, Average Distance: 0.219154


In [88]:
#Assign a label to each Centroid
# count total num of labels per centroid, using the label of each training sample to their closest centroid
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]

    #idx length is the number of samples, idx[i] indicates the clusters label(in range 0 - 24)
# Assign the most freunt label to each centriod
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

In [89]:
# Evaluation ops
#LookUp : centroid -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)

# Compute Accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32)) #  axis row
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Test Model
test_x, test_y, = mnist.test.images, mnist.test.labels
print("Test Accuracy:", sess.run(accuracy_op, feed_dict = {X : test_x, Y : test_y}))

Test Accuracy: 0.7127
